Chapter 11. Image Processing and Text Recognition
===

In [2]:
from PIL import Image, ImageFilter

kitten = Image.open("page.jpg")
blurryKitten = kitten.filter(ImageFilter.GaussianBlur)
blurryKitten.save("kitten_blurred.jpg")
blurryKitten.show()

In [23]:
from PIL import Image
import subprocess

def cleanFile(filePath, newFilePath):
    image = Image.open(filePath)

    #Set a threshold value for the image, and save
    image = image.point(lambda x: 0 if x<143 else 255)
    image.save(newFilePath)

    #call tesseract to do OCR on the newly created image
       subprocess.call(["tesseract", newFilePath, "output"])

    #Open and read the resulting data file
    outputFile = open("output.txt", 'r')
    print("Text Captured:\n", outputFile.read())
    outputFile.close()


In [ ]:
cleanFile("page.jpg", "text_2_clean.jpg")

PNG recognition
---
Original image
<img src="ch4-1.png" />
Handled Image
<img src="ch4-1_clean.png" />

In [6]:
cleanFile("ch4-1.png", "ch4-1_clean.png")

Text Captured:
 f (E) /
f ’(E) +




Tradtional Chinese Image
---
<img src="TradChinese.png" width=40%/>

In [26]:
!tesseract TradChinese.png output -l chi_tra

Tesseract Open Source OCR Engine v3.05.00dev with Leptonica
Detected 97 diacritics


In [27]:
!cat output.txt

 

 

「趟客鰻胡纓，吳鉤霜雪明。銀鞍照ˊ …u
十步殺_人 千裹不留行。 辜菖了…弗衣去

 

 

 

 

閑過信陵飲 脫劍膝前橫。 將炙口炎朱亥
三杯吐然諾 五嶽到為輕。 眼花=熱後
求叉趟揮金鍾 邯鄲先震驚。 千秋二 壯‥‥
縱死俠骨香 ， 不慚世上英。 誰能書閣ˉ‥

 

 

 

 

 

 

，瘋還如流星。

深藏身與名。
持觴勸侯羸。
意氣素霓生 。
姮赫大梁城 。
ˊ首人玄經?

 

 

 

 

 

 

」



Why message worse
---
Here, the text in the image was created by $\TeX$ (and in image format) which is differeent from general text come from font file. 

In [ ]:
import time
from urllib.request import urlretrieve
import subprocess
from selenium import webdriver

#driver = webdriver.PhantomJS(executable_path='/Users/ryan/Documents/pythonscraping/code/headless/phantomjs-1.9.8-macosx/bin/phantomjs')
driver = webdriver.Firefox()
driver.get("http://www.amazon.com/War-Peace-Leo-Nikolayevich-Tolstoy/dp/1427030200")
time.sleep(2)

driver.find_element_by_id("img-canvas").click()
#The easiest way to get exactly one of every page
imageList = set()

#Wait for the page to load
time.sleep(10)
print(driver.find_element_by_id("sitbReaderRightPageTurner").get_attribute("style"))
while "pointer" in driver.find_element_by_id("sitbReaderRightPageTurner").get_attribute("style"):
    #While we can click on the right arrow, move through the pages
    driver.find_element_by_id("sitbReaderRightPageTurner").click()
    time.sleep(2)
    #Get any new pages that have loaded (multiple pages can load at once)
    pages = driver.find_elements_by_xpath("//div[@class='pageImage']/div/img")
    for page in pages:
        image = page.get_attribute("src")
        imageList.add(image)

driver.quit()

#Start processing the images we've collected URLs for with Tesseract
for image in sorted(imageList):
    urlretrieve(image, "page.jpg")
    p = subprocess.Popen(["tesseract", "page.jpg", "page"], stdout=subprocess.PIPE,stderr=subprocess.PIPE)
    p.wait()
    f = open("page.txt", "r")
    print(f.read())

In [ ]:
from urllib.request import urlretrieve
from urllib.request import urlopen
from bs4 import BeautifulSoup
import subprocess
import requests
from PIL import Image
from PIL import ImageOps

def cleanImage(imagePath):
    image = Image.open(imagePath)
    image = image.point(lambda x: 0 if x<143 else 255)
    borderImage = ImageOps.expand(image,border=20,fill='white')
    borderImage.save(imagePath)

html = urlopen("http://www.pythonscraping.com/humans-only")
bsObj = BeautifulSoup(html, "html.parser")
#Gather prepopulated form values
imageLocation = bsObj.find("img", {"title": "Image CAPTCHA"})["src"]
formBuildId = bsObj.find("input", {"name":"form_build_id"})["value"]
captchaSid = bsObj.find("input", {"name":"captcha_sid"})["value"]
captchaToken = bsObj.find("input", {"name":"captcha_token"})["value"]

captchaUrl = "http://pythonscraping.com"+imageLocation
urlretrieve(captchaUrl, "captcha.jpg")
cleanImage("captcha.jpg")
p = subprocess.Popen(["tesseract", "captcha.jpg", "captcha"], stdout=
    subprocess.PIPE,stderr=subprocess.PIPE)
p.wait()
f = open("captcha.txt", "r")

#Clean any whitespace characters
captchaResponse = f.read().replace(" ", "").replace("\n", "")
print("Captcha solution attempt: "+captchaResponse)

if len(captchaResponse) == 5:
    params = {"captcha_token":captchaToken, "captcha_sid":captchaSid,   
              "form_id":"comment_node_page_form", "form_build_id": formBuildId, 
                  "captcha_response":captchaResponse, "name":"Ryan Mitchell", 
                  "subject": "I come to seek the Grail", 
                  "comment_body[und][0][value]": 
                                           "...and I am definitely not a bot"}
    r = requests.post("http://www.pythonscraping.com/comment/reply/10", 
                          data=params)
    responseObj = BeautifulSoup(r.text)
    if responseObj.find("div", {"class":"messages"}) is not None:
        print(responseObj.find("div", {"class":"messages"}).get_text())
else:
    print("There was a problem reading the CAPTCHA correctly!")